In [3]:
# train_emotion.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os

# -------------------------
# Config
# -------------------------
# Use portable path for notebooks
data_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "src", "data"))
batch_size = 64
num_epochs = 20
num_classes = 7
learning_rate = 0.001
save_path = "Final_ModelV2.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# -------------------------
# Data transforms
# -------------------------
transform = {
    "train": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}

# -------------------------
# Datasets and Loaders
# -------------------------
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform["train"])
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform["test"])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# -------------------------
# Model (ResNet18 pretrained on ImageNet)
# -------------------------
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # replace final layer

model = model.to(device)

# -------------------------
# Loss and Optimizer
# -------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# -------------------------
# Training loop
# -------------------------
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss/len(train_loader):.4f} Acc: {acc:.2f}%")

# -------------------------
# Save model
# -------------------------
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")


cpu


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\USER\\Desktop\\src\\data\\train'

## Improved Training Version

✅ Validation loop each epoch
✅ Learning rate scheduler
✅ Stronger data augmentation
✅ Save best model only
✅ Mixed precision training (if CUDA available)

In [ ]:
# train_emotion_v2.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os

# -------------------------
# Config
# -------------------------
data_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "src", "data"))   # <-- change to your dataset folder
batch_size = 64
num_epochs = 20
num_classes = 7
learning_rate = 0.001
save_path = "Final_ModelV3.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------------
# Data transforms
# -------------------------
transform = {
    "train": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}

# -------------------------
# Datasets and Loaders
# -------------------------
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform["train"])
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform["test"])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# -------------------------
# Model (ResNet18 pretrained on ImageNet)
# -------------------------
model = models.resnet18(pretrained=True) # weights=ResNet18_Weights.IMAGENET1K_V1
model.fc = nn.Linear(model.fc.in_features, num_classes)  # replace final layer

model = model.to(device)

# -------------------------
# Loss, Optimizer, Scheduler
# -------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Mixed precision scaler
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))

# -------------------------
# Training + Validation loop
# -------------------------
best_acc = 0.0

for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    train_loss = running_loss / len(train_loader)

    # ---- Validation ----
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    val_loss = val_loss / len(test_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% "
          f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # ---- Save best model ----
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"✅ Best model saved with acc {best_acc:.2f}%")

    scheduler.step()

print(f"Training complete. Best Val Acc: {best_acc:.2f}%")


Using device: cuda


c:\Users\HWA\Desktop\AI Project UV\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HWA\Desktop\AI Project UV\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\HWA\AppData\Local\Temp\ipykernel_8096\605146182.py:68: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))
C:\Users\HWA\AppData\Local\Temp\ipykernel_8096\605146182.py:84: FutureWarning:

Epoch [1/20] Train Loss: 1.2436 | Train Acc: 52.62% | Val Loss: 1.2894 | Val Acc: 54.15%
✅ Best model saved with acc 54.15%
Epoch [2/20] Train Loss: 1.0649 | Train Acc: 60.01% | Val Loss: 1.1361 | Val Acc: 58.41%
✅ Best model saved with acc 58.41%
Epoch [3/20] Train Loss: 0.9949 | Train Acc: 62.36% | Val Loss: 1.0221 | Val Acc: 62.44%
✅ Best model saved with acc 62.44%
Epoch [4/20] Train Loss: 0.9389 | Train Acc: 64.90% | Val Loss: 0.9820 | Val Acc: 63.47%
✅ Best model saved with acc 63.47%
Epoch [5/20] Train Loss: 0.8973 | Train Acc: 66.31% | Val Loss: 0.9690 | Val Acc: 64.45%
✅ Best model saved with acc 64.45%
Epoch [6/20] Train Loss: 0.8476 | Train Acc: 68.33% | Val Loss: 0.9706 | Val Acc: 64.14%
Epoch [7/20] Train Loss: 0.8127 | Train Acc: 69.47% | Val Loss: 0.9657 | Val Acc: 63.89%
Epoch [8/20] Train Loss: 0.6484 | Train Acc: 76.04% | Val Loss: 0.8831 | Val Acc: 67.97%
✅ Best model saved with acc 67.97%
Epoch [9/20] Train Loss: 0.5817 | Train Acc: 78.67% | Val Loss: 0.8915 | Val A